In [1]:
import os 
import glob
import random
import numpy as np 
import pandas as pd 
import SimpleITK as sitk
from keras.optimizers import Adam
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import model.metrics as mm
import model.callBack as mc
from model.network import unet3d
from utils.utils import *
from utils.getTrainData import image_gen3d
import configTrain

Using TensorFlow backend.


In [2]:
folder = '/public/lixin/lung/lung-lobe-seg/data/'

In [3]:
img_list = glob.glob(os.path.join(folder, '*_img.nii.gz'))

In [4]:
len(img_list)

47

In [5]:
train_list = img_list[:37]
val_list = img_list[37:]

In [6]:
len(val_list)

10

In [7]:
aug_gen = image_gen3d(train_list) 
valid_x, valid_y = next(image_gen3d(val_list, len(val_list))) #

In [8]:
model = unet3d((48,224,224,1))

In [9]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 48, 224, 224, 0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 48, 224, 224, 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 48, 224, 224, 64          conv3d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 48, 224, 224, 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv3d_2 (

# Train

In [17]:
callbacks_list = call(
         weight_path = configTrain.best_model_save_path, 
         monitor = configTrain.monitor , 
         mode = configTrain.mode, 
         reduce_lr_p = configTrain.reduce_lr_p, 
         early_p = configTrain.early_p, 
         log_csv_path = configTrain.log_csv_path,
         min_lr = configTrain.min_lr,
         factor = 0.5,
         min_delta = 0.0001,
         cooldown = 2,
         verbose = 1)

/opt/anaconda3/envs/py36_tensorflow-1.12.2/lib/python3.6/site-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [10]:
n_labels = 6
label_wise_dice_metrics = [mm.get_label_dice_coefficient_function(index) for index in range(n_labels)]
metrics = label_wise_dice_metrics

In [19]:
import segmentation_models as sm
# from keras.utils import multi_gpu_model
# muti_model = multi_gpu_model(model, gpus=3) 
model.load_weights(configTrain.best_model_save_path) 

model.compile(
    optimizer = Adam(lr = configTrain.lr),
    loss = mm.dice_coefficient_loss, #sm.losses.bce_jaccard_loss,
    metrics = metrics#[sm.metrics.iou_score],
)

loss_history = [model.fit_generator(
                        aug_gen, 
                        steps_per_epoch = 20, 
                        epochs = 200, 
                        validation_data = (valid_x, valid_y),
                        callbacks = callbacks_list,
                        workers = 1 
                                  )]

Segmentation Models: using `keras` framework.
Epoch 1/200
20/20 [==============================] - 77s 4s/step - loss: -0.9857 - label_0_dice_coef: 0.9853 - label_1_dice_coef: 0.9875 - label_2_dice_coef: 0.9888 - label_3_dice_coef: 0.9890 - label_4_dice_coef: 0.9882 - label_5_dice_coef: 0.9884 - val_loss: -0.9788 - val_label_0_dice_coef: 0.9757 - val_label_1_dice_coef: 0.9783 - val_label_2_dice_coef: 0.9794 - val_label_3_dice_coef: 0.9790 - val_label_4_dice_coef: 0.9803 - val_label_5_dice_coef: 0.9805

Epoch 00001: val_loss improved from inf to -0.97878, saving model to model_save/2019-10-16-2/one/pretrain_model_weights.best.hdf5
Epoch 2/200
20/20 [==============================] - 50s 2s/step - loss: -0.9864 - label_0_dice_coef: 0.9824 - label_1_dice_coef: 0.9862 - label_2_dice_coef: 0.9877 - label_3_dice_coef: 0.9885 - label_4_dice_coef: 0.9889 - label_5_dice_coef: 0.9892 - val_loss: -0.9788 - val_label_0_dice_coef: 0.9775 - val_label_1_dice_coef: 0.9809 - val_label_2_dice_coef: 0.98

KeyboardInterrupt: 

In [20]:
model.load_weights(configTrain.best_model_save_path) 
model.save(configTrain.model_save_path)
print('Done!')

Done!
